<a href="https://colab.research.google.com/github/HoarfrostRaven/BigData/blob/main/BGProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare

In [1]:
!wget https://snap.stanford.edu/data/web-Google.txt.gz
!gzip -d web-Google.txt.gz

--2025-03-29 13:01:08--  https://snap.stanford.edu/data/web-Google.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21168784 (20M) [application/x-gzip]
Saving to: ‘web-Google.txt.gz’

web-Google.txt.gz   100%[===================>]  20.19M  5.72MB/s    in 3.5s    

2025-03-29 13:01:12 (5.72 MB/s) - ‘web-Google.txt.gz’ saved [21168784/21168784]



In [ ]:
! pwd

/content


In [ ]:
! ls

sample_data  web-Google.txt  web-Google.txt.gz


In [2]:
# !pip install pyspark
import random
from pyspark import SparkConf
from pyspark import StorageLevel
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

## CCF

In [5]:
def connected_components_ccf(data, num_partitions = 1):
    """
    Computes connected components using the CCF algorithm.

    Args:
        data: An RDD of edges represented as tuples (node1, node2).
              Example: sc.parallelize([(1, 2), (2, 3), (2, 4), (4, 5), (6, 7), (7, 8)])
        num_partitions: Number of partitions to optimize parallelism.

    Returns:
        An RDD of edges representing the connected components.
        Example: sc.parallelize([(2, 1), (3, 1), (4, 1), (5, 1), (7, 6), (8, 6)])
    """

    # Initialize bidirectional edges with partition optimization
    edges = (data.flatMap(lambda x: [(x[0], x[1]), (x[1], x[0])])
             .distinct()
             .repartition(num_partitions)
             .persist(StorageLevel.MEMORY_AND_DISK))  # Spill to disk if OOM

    converged = False
    iteration = 0
    prev_edges = None  # Track previous iteration's data

    while not converged:
        iteration += 1
        print(f"--- Iteration {iteration} ---")

        # Filter first to reduce data volume before reduce
        filtered = edges.filter(lambda x: x[1] < x[0]).cache()

        # Compute minimum neighbors with partition preservation
        min_values = filtered.reduceByKey(min, numPartitions=num_partitions).cache()
        filtered.unpersist()  # Release intermediate data immediately

        # Join with partition alignment to minimize shuffle
        new_edges = (min_values.join(edges, numPartitions=num_partitions)
                     .filter(lambda x: x[1][0] != x[1][1])  # Remove self-edges
                     .map(lambda x: (x[1][1], x[1][0]))     # Remap edges
                     .cache())

        if new_edges.isEmpty():
            converged = True
            result = min_values
        else:
            # Release data from two iterations back
            if prev_edges is not None:
                prev_edges.unpersist()

            # Track current edges for future cleanup
            prev_edges = edges

            # Update edges with partition optimization
            edges = (min_values.union(new_edges)
                     .flatMap(lambda x: [(x[0], x[1]), (x[1], x[0])])
                     .distinct()
                     .repartition(num_partitions)
                     .persist(StorageLevel.MEMORY_AND_DISK))

            # Release current iteration's intermediates
            min_values.unpersist()
            new_edges.unpersist()

    # Final cleanup
    edges.unpersist()
    return result

## CCF Sorting

In [ ]:
def connected_components_ccf_sorting(data, num_partitions=1):
    """
    Computes connected components using the CCF algorithm (with sorting instead of reduceByKey).

    Args:
        data: An RDD of edges represented as tuples (node1, node2).
        num_partitions: Number of partitions to optimize parallelism.

    Returns:
        An RDD of edges representing the connected components.
    """

    # Initialize bidirectional edges with partition optimization
    edges = (data.flatMap(lambda x: [(x[0], x[1]), (x[1], x[0])])
             .distinct()
             .repartition(num_partitions)
             .persist(StorageLevel.MEMORY_AND_DISK))  # Spill to disk if OOM

    converged = False
    iteration = 0
    prev_edges = None  # Track previous iteration's data

    while not converged:
        iteration += 1
        print(f"--- Iteration {iteration} ---")

        # Filter to keep (higher_node, lower_node) pairs
        filtered = edges.filter(lambda x: x[1] < x[0]).cache()

        # Compute minimum neighbor using sorting
        min_values = (filtered.groupByKey(num_partitions)
                      .mapValues(lambda neighbors: sorted(neighbors)[0])  # Sort and take min
                      .cache())

        filtered.unpersist()  # Release intermediate data immediately

        # Join with partition alignment to minimize shuffle
        new_edges = (min_values.join(edges, num_partitions)
                     .filter(lambda x: x[1][0] != x[1][1])  # Remove self-edges
                     .map(lambda x: (x[1][1], x[1][0]))     # Remap edges
                     .cache())

        if new_edges.isEmpty():
            converged = True
            result = min_values
        else:
            # Release data from two iterations back
            if prev_edges is not None:
                prev_edges.unpersist()

            # Track current edges for future cleanup
            prev_edges = edges

            # Update edges with partition optimization
            edges = (min_values.union(new_edges)
                     .flatMap(lambda x: [(x[0], x[1]), (x[1], x[0])])
                     .distinct()
                     .repartition(num_partitions)
                     .persist(StorageLevel.MEMORY_AND_DISK))

            # Release current iteration's intermediates
            min_values.unpersist()
            new_edges.unpersist()

    # Final cleanup
    edges.unpersist()
    return result


## 计算 num_partitions

In [7]:
def estimate_rdd_size_gb(rdd, sample_fraction=0.01):
    """
    估算 RDD 数据的大小（GB），通过采样计算平均行大小并推测整体数据大小。

    :param rdd: 要估算的 Spark RDD
    :param sample_fraction: 采样比例（默认 1%），用于计算平均行大小
    :return: 估算的 RDD 大小（GB）
    """
    from pyspark import StorageLevel

    # 采样 RDD（非放回抽样）
    sample_data = rdd.sample(False, sample_fraction)

    # 计算平均每行字节大小（转换为字符串后编码为 UTF-8 字节）
    avg_line_size = sample_data.map(lambda x: len(str(x).encode('utf-8'))).mean()

    # 计算 RDD 总行数
    num_lines = rdd.count()

    # 估算 RDD 总大小（字节）
    data_size_bytes = num_lines * avg_line_size

    # 转换为 GB
    data_size_gb = data_size_bytes / (1024 ** 3)

    print(f"Estimated RDD Size: {data_size_gb:.4f} GB")
    return data_size_gb

In [6]:
def get_optimal_num_partitions(sc, data_size_gb=None):
    """
    计算 Spark 任务的最佳分区数
    :param sc: SparkContext
    :param data_size_gb: 数据大小 (GB)，可选
    :return: 计算得到的最佳分区数
    """
    # 获取可用 CPU 核心数
    available_cores = sc.defaultParallelism  # 等价于 os.cpu_count() 或 psutil.cpu_count(logical=True)

    # 基于 CPU 核心数的推荐分区数
    cpu_based_partitions = 2 * available_cores

    # 基于数据量的分区数（假设每个分区 128MB）
    if data_size_gb:
        partition_size_mb = 128  # 假设单个分区最佳大小 128MB
        data_size_mb = data_size_gb * 1024
        data_based_partitions = max(1, int(data_size_mb / partition_size_mb))
    else:
        data_based_partitions = cpu_based_partitions  # 无数据大小时默认使用 CPU 计算

    # 取两者中的较大值，确保计算不会受限
    optimal_partitions = max(cpu_based_partitions, data_based_partitions)

    print(f"Optimal Partitions: {optimal_partitions} (CPU-based: {cpu_based_partitions}, Data-based: {data_based_partitions})")
    return optimal_partitions

## 代码测试

In [ ]:
# Example data for testing
test_data = sc.parallelize([(1, 2), (2, 3), (2, 4), (4, 5), (6, 7), (7, 8)])

--- Iteration 1 ---
--- Iteration 2 ---
--- Iteration 3 ---
--- Iteration 4 ---
Connected components: [(4, 1), (2, 1), (3, 1), (5, 1), (7, 6), (8, 6)]


In [ ]:
# Run the connected_components_ccf algorithm
result = connected_components_ccf(test_data)

# Collect and print the result
print("Connected components:", result.collect())

In [ ]:
# Run the connected_components_ccf_sorting algorithm
result = connected_components_ccf_sorting(test_data)

# Collect and print the result
print("Connected components:", result.collect())

## 大数据实验

In [3]:
# Load the file, ignore first 4 lines and split the data
data = (
    sc.textFile("/content/web-Google.txt")
    .filter(lambda line: line.strip() and not line.startswith('#'))
    .map(lambda line: tuple(map(int, line.split())))
)

In [4]:
print(data.take(5))

[(0, 11342), (0, 824020), (0, 867923), (0, 891835), (11342, 0)]


In [8]:
data_size_gb = estimate_rdd_size_gb(data, sample_fraction=0.01)
num_partitions = get_optimal_num_partitions(sc, data_size_gb)

print(f"Estimated RDD Size: {data_size_gb:.4f} GB")
print(f"Optimal Partitions: {num_partitions}")

Estimated RDD Size: 0.0750 GB
Optimal Partitions: 4 (CPU-based: 4, Data-based: 1)
Estimated RDD Size: 0.0750 GB
Optimal Partitions: 4


In [ ]:
# Compute connected components using the CCF algorithm
connected_components = connected_components_ccf(data, num_partitions)

In [ ]:
# Print the results
print(connected_components.collect())

In [ ]:
# Compute connected components using the CCF-Sorting algorithm
connected_components_sorting = connected_components_ccf_sorting(data, num_partitions)

In [ ]:
# Print the results
print(connected_components_sorting.collect())

## 随即图实验

In [ ]:
def generate_random_graph(num_nodes, num_edges, seed=None):
    """
    生成一个随机无向图，返回边列表，格式为 (int, int)。

    :param num_nodes: 节点数量
    :param num_edges: 边的数量
    :param seed: 随机种子（可选，保证可复现）
    :return: 由 (node1, node2) 组成的边列表
    """
    if seed is not None:
        random.seed(seed)

    edges = set()

    while len(edges) < num_edges:
        node1 = random.randint(0, num_nodes - 1)
        node2 = random.randint(0, num_nodes - 1)

        # 确保无自环、无重复边（无向图存储较小节点在前）
        if node1 != node2:
            edge = (min(node1, node2), max(node1, node2))
            edges.add(edge)

    return list(edges)

### Test1

In [ ]:
# Data for testing 1
random_graph_1 = generate_random_graph(10000, 50000)
random_graph_1_rdd = sc.parallelize(random_graph_1)

In [ ]:
data_size_gb = estimate_rdd_size_gb(random_graph_1_rdd, sample_fraction=0.01)
num_partitions = get_optimal_num_partitions(sc, data_size_gb)

print(f"Estimated RDD Size: {data_size_gb:.4f} GB")
print(f"Optimal Partitions: {num_partitions}")

In [ ]:
# Run the connected_components_ccf algorithm
random_graph_1_result = connected_components_ccf(random_graph_1_rdd, num_partitions)

In [ ]:
# Collect and print the result
print("Connected components:", random_graph_1_result.collect())

In [ ]:
# Run the connected_components_ccf algorithm
random_graph_1_result_sorting = connected_components_ccf_sorting(random_graph_1_rdd, num_partitions)

In [ ]:
# Collect and print the result
print("Connected components:", random_graph_1_result_sorting.collect())

### Test 2

In [ ]:
# Data for testing 2
random_graph_2 = generate_random_graph(100000, 1000000)
random_graph_2_rdd = sc.parallelize(random_graph_2)

In [ ]:
data_size_gb = estimate_rdd_size_gb(random_graph_2_rdd, sample_fraction=0.01)
num_partitions = get_optimal_num_partitions(sc, data_size_gb)

print(f"Estimated RDD Size: {data_size_gb:.4f} GB")
print(f"Optimal Partitions: {num_partitions}")

In [ ]:
# Run the connected_components_ccf algorithm
random_graph_2_result = connected_components_ccf(random_graph_2_rdd, num_partitions)

In [ ]:
# Collect and print the result
print("Connected components:", random_graph_2_result.collect())

In [ ]:
# Run the connected_components_ccf algorithm
random_graph_2_result_sorting = connected_components_ccf_sorting(random_graph_2_rdd, num_partitions)

In [ ]:
# Collect and print the result
print("Connected components:", random_graph_2_result_sorting.collect())

### Test 3

In [ ]:
# Data for testing 3
random_graph_3 = generate_random_graph(10000000, 50000000)
random_graph_3_rdd = sc.parallelize(random_graph_3)

In [ ]:
data_size_gb = estimate_rdd_size_gb(random_graph_3_rdd, sample_fraction=0.01)
num_partitions = get_optimal_num_partitions(sc, data_size_gb)

print(f"Estimated RDD Size: {data_size_gb:.4f} GB")
print(f"Optimal Partitions: {num_partitions}")

In [ ]:
# Run the connected_components_ccf algorithm
random_graph_3_result = connected_components_ccf(random_graph_3_rdd, num_partitions)

In [ ]:
# Collect and print the result
print("Connected components:", random_graph_3_result.collect())

In [ ]:
# Run the connected_components_ccf algorithm
random_graph_3_result_sorting = connected_components_ccf_sorting(random_graph_3_rdd, num_partitions)

In [ ]:
# Collect and print the result
print("Connected components:", random_graph_3_result_sorting.collect())